In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

from astropy.table import Table
import pyvo as vo
print(f"py{vo.__version__=}")

def pprint_columns_description(tablename):
    """Pretty print column description table"""
    rows = []
    for col in simbadtap.run_sync(
        "select top 0 * from {}".format(tablename)).to_table().itercols():
        rows.append([col.name, col.dtype.str, col.description])
    tt = Table(rows=rows, names=['colname', 'dtype','description'])
    return tt

def custom_agg(x):
    if len(set(x)) == 1:  # Check if all values are the same
        return x.iloc[0]  # If yes, return a single value
    else:
        return x.tolist()  # If not, return a list



pyvo.__version__='1.4.1'


In [13]:
simbadtap = vo.dal.TAPService("http://simbad.u-strasbg.fr/simbad/sim-tap")

In [29]:
d = pd.read_csv('full_str',delim_whitespace=True)
d

,source_id,TIC,RA~JK,DEC~JK
0,3517127900988821504,5816844,187.34692,-18.13551
1,1495329392800826624,23746001,208.24585,36.92707
2,1879544900675463552,27782233,332.51130,25.06619
3,4653674069685812864,33770579,63.46109,-72.31437
4,3600739335011156992,35020718,179.84178,-4.02409
...,...,...,...,...
95,1128175247373543168,142491300,161.77070,75.73971
96,1948563616375288960,162239156,331.33941,35.55149
97,6785706005902423680,209393544,325.48672,-29.27517
98,3145755479203737856,320417198,117.94643,9.42449


In [30]:
d['source_id']

0     3517127900988821504
1     1495329392800826624
2     1879544900675463552
3     4653674069685812864
4     3600739335011156992
             ...         
95    1128175247373543168
96    1948563616375288960
97    6785706005902423680
98    3145755479203737856
99      19200191735883008
Name: source_id, Length: 100, dtype: int64

In [15]:
result = simbadtap.run_sync("select * from TAP_SCHEMA.tables where schema_name not like 'TAP_SCHEMA'")
print("number of tables available = {:d}".format(len(result.to_table())))
result.to_table()['table_name','description']

number of tables available = 30


table_name,description
object,object
basic,General data about an astronomical object
ids,all names concatenated with pipe
alltypes,all object types concatenated with pipe
ident,Identifiers of an astronomical object
cat,Catalogues name
flux,Magnitude/Flux information about an astronomical object
...,...
mesISO,Infrared Space Observatory (ISO) observing log.
mesFe_h,"Collection of metallicity, as well as Teff, logg for stars."


In [33]:
ful_result = pd.DataFrame()


total_iterations = len(d['TIC'])


for i in tqdm(d['source_id'], total=total_iterations, desc='Processing TICs'):
    query = f"""
    SELECT
  b.main_id,
  b.otype,
  b.otype_txt,
  i.id AS "GAIA",
  ot.label AS "Other type",
  mesSpT.oidref,
  mesSpT.sptype,
  mesSpT.bibcode,
  mesFe_H.log_g,
  mesFe_H.teff,
  mesFe_H.bibcode,
  mesFe_H.oidref
FROM basic AS b
JOIN ident AS i ON b.oid = i.oidref
JOIN otypedef AS ot ON b.otype = ot.otype
LEFT JOIN mesSpT USING(oidref)
LEFT JOIN mesFe_H USING(oidref)
WHERE id = 'Gaia DR3{i}'
ORDER BY id;
    """
    result = simbadtap.run_sync(query)
    
    
    ful_result = pd.concat([ful_result, result.to_table().to_pandas()], ignore_index=True)


#ful_result = ful_result.drop(columns=['oidref'])


#ful_result = ful_result[['id'] + [col for col in ful_result.columns if col != 'id']]
ful_result

Processing TICs: 100%|████████████████████████| 100/100 [00:07<00:00, 13.07it/s]


,main_id,otype,otype_txt,GAIA,Other type,oidref,sptype,bibcode,log_g,teff,bibcode2,oidref2
0,EC 12267-1751,HS*,HS*,Gaia DR3 3517127900988821504,HotSubdwarf,1946629,sdO,1997MNRAS.287..867K,NaN,<NA>,,<NA>
1,PG 1350+372,HS*,HS*,Gaia DR3 1495329392800826624,HotSubdwarf,2211327,sdB/sdO,2021ApJS..256...28L,6.46,29987,2021ApJS..256...28L,2211327
2,PG 1350+372,HS*,HS*,Gaia DR3 1495329392800826624,HotSubdwarf,2211327,sdOA,2008Ap.....51..226S,6.46,29987,2021ApJS..256...28L,2211327
3,PG 1350+372,HS*,HS*,Gaia DR3 1495329392800826624,HotSubdwarf,2211327,sdOA,2008AJ....136..946M,6.46,29987,2021ApJS..256...28L,2211327
4,PG 1350+372,HS*,HS*,Gaia DR3 1495329392800826624,HotSubdwarf,2211327,sdB,1986ApJS...61..305G,6.46,29987,2021ApJS..256...28L,2211327
...,...,...,...,...,...,...,...,...,...,...,...,...
337,GALEX J075147.0+092526,HS*,HS*,Gaia DR3 3145755479203737856,HotSubdwarf,7834568,sdB,2012MNRAS.427.2180N,5.65,29890,2011MNRAS.410.2095V,7834568
338,PG 0229+064,HS*,HS*,Gaia DR3 19200191735883008,HotSubdwarf,1417553,sdB3VHe13,2013A&A...551A..31D,4.55,19000,2013A&A...551A..31D,1417553
339,PG 0229+064,HS*,HS*,Gaia DR3 19200191735883008,HotSubdwarf,1417553,B,1994ApJ...432..351S,4.55,19000,2013A&A...551A..31D,1417553
340,PG 0229+064,HS*,HS*,Gaia DR3 19200191735883008,HotSubdwarf,1417553,B,1990A&AS...86...53M,4.55,19000,2013A&A...551A..31D,1417553


In [34]:
tic_column = ful_result['GAIA']

# Drop the 'TIC' column from the DataFrame
ful_result = ful_result.drop(columns=['GAIA'])

# Concatenate the 'TIC' column at the beginning
ful_result = pd.concat([tic_column, ful_result], axis=1)
ful_result

,GAIA,main_id,otype,otype_txt,Other type,oidref,sptype,bibcode,log_g,teff,bibcode2,oidref2
0,Gaia DR3 3517127900988821504,EC 12267-1751,HS*,HS*,HotSubdwarf,1946629,sdO,1997MNRAS.287..867K,NaN,<NA>,,<NA>
1,Gaia DR3 1495329392800826624,PG 1350+372,HS*,HS*,HotSubdwarf,2211327,sdB/sdO,2021ApJS..256...28L,6.46,29987,2021ApJS..256...28L,2211327
2,Gaia DR3 1495329392800826624,PG 1350+372,HS*,HS*,HotSubdwarf,2211327,sdOA,2008Ap.....51..226S,6.46,29987,2021ApJS..256...28L,2211327
3,Gaia DR3 1495329392800826624,PG 1350+372,HS*,HS*,HotSubdwarf,2211327,sdOA,2008AJ....136..946M,6.46,29987,2021ApJS..256...28L,2211327
4,Gaia DR3 1495329392800826624,PG 1350+372,HS*,HS*,HotSubdwarf,2211327,sdB,1986ApJS...61..305G,6.46,29987,2021ApJS..256...28L,2211327
...,...,...,...,...,...,...,...,...,...,...,...,...
337,Gaia DR3 3145755479203737856,GALEX J075147.0+092526,HS*,HS*,HotSubdwarf,7834568,sdB,2012MNRAS.427.2180N,5.65,29890,2011MNRAS.410.2095V,7834568
338,Gaia DR3 19200191735883008,PG 0229+064,HS*,HS*,HotSubdwarf,1417553,sdB3VHe13,2013A&A...551A..31D,4.55,19000,2013A&A...551A..31D,1417553
339,Gaia DR3 19200191735883008,PG 0229+064,HS*,HS*,HotSubdwarf,1417553,B,1994ApJ...432..351S,4.55,19000,2013A&A...551A..31D,1417553
340,Gaia DR3 19200191735883008,PG 0229+064,HS*,HS*,HotSubdwarf,1417553,B,1990A&AS...86...53M,4.55,19000,2013A&A...551A..31D,1417553


In [35]:
ful_result_grouped = ful_result.groupby('GAIA').agg(custom_agg).reset_index()

# Display the grouped DataFrame
ful_result_grouped


,GAIA,main_id,otype,otype_txt,Other type,oidref,sptype,bibcode,log_g,teff,bibcode2,oidref2
0,Gaia DR3 1094879011629873920,PG 0749+658,HS*,HS*,HotSubdwarf,394297,"[sdB , sdOB ...","[1994ApJ...432..351S, 1986ApJS...61..305G, 198...","[nan, nan, nan, nan]",<NA>,,<NA>
1,Gaia DR3 1128175247373543168,PG 1043+760,HS*,HS*,HotSubdwarf,391926,sdB,"[1986ApJS...61..305G, 2008AJ....136..946M]",5.39,27600,2015A&A...576A..44K,391926
2,Gaia DR3 141419254284409472,ATO J042.2703+36.7306,V*,V*,Variable*,<NA>,,,NaN,<NA>,,<NA>
3,Gaia DR3 1495329392800826624,PG 1350+372,HS*,HS*,HotSubdwarf,2211327,"[sdB/sdO , sdOA ...","[2021ApJS..256...28L, 2008Ap.....51..226S, 200...",6.46,29987,2021ApJS..256...28L,2211327
4,Gaia DR3 1695662021992833920,2MASS J15292631+7011543,HS?,HS?,"[HotSubdwarf_Candidate, ]",<NA>,,,"[nan, nan]",<NA>,,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Gaia DR3 680914734899282560,GALEX J080628.6+242058,HS*,HS*,HotSubdwarf,8344543,"[sdB/sdO , sdB/sdO...","[2021ApJS..256...28L, 2021ApJS..256...28L, 202...","[5.289999961853027, 5.510000228881836, 5.5, 5....","[27934, 28391, 28190, 27990, 27934, 28391, 281...","[2021ApJS..256...28L, 2019ApJ...881....7L, 201...",8344543
96,Gaia DR3 77515058657678592,CHSS 3497,HS*,HS*,HotSubdwarf,5265521,"[sdB/sdO , sdB/sdO...","[2021ApJS..256...28L, 2021ApJS..256...28L, 202...","[6.320000171661377, 5.789999961853027, 5.76999...","[29589, 25448, 30310, 29589, 25448, 30310, 295...","[2021ApJS..256...28L, 2019ApJ...881....7L, 201...",5265521
97,Gaia DR3 890152695314991488,KUV 07088+3232,HS*,HS*,HotSubdwarf,951707,sdO,1987AJ.....94.1271W,NaN,<NA>,,<NA>
98,Gaia DR3 944892878136403712,FBS 0639+391,HS*,HS*,HotSubdwarf,898955,"[sdB/sdO , sdB/sdO...","[2021ApJS..256...28L, 2021ApJS..256...28L, 201...","[5.53000020980835, 5.570000171661377, 5.530000...","[29658, 29133, 29658, 29133, 29658, 29133]","[2021ApJS..256...28L, 2019ApJ...881....7L, 202...",898955


In [38]:
ful_result_grouped.to_excel('ful_star_rslts.xlsx')

In [37]:
# Assuming d is your original dataframe with TIC values
# Assuming ful_result_grouped is your grouped dataframe

# Get unique TIC values from d
unique_tic_d = d['TIC'].unique()

# Create a dataframe with only TIC values from d
tic_df = pd.DataFrame({'TIC': unique_tic_d})

# Remove non-numeric parts from the TIC column in ful_result_grouped
ful_result_grouped['TIC'] = ful_result_grouped['TIC'].str.replace('TIC ', '').astype(int)

# Merge tic_df with ful_result_grouped using a left join
merged_df = pd.merge(tic_df, ful_result_grouped, on='TIC', how='left')

# Display the merged dataframe
merged_df


KeyError: 'TIC'

In [26]:
merged_df.to_excel('ful_star_rslts.xlsx')

In [27]:
tic_df

,TIC
0,5816844
1,23746001
2,27782233
3,33770579
4,35020718
...,...
95,142491300
96,162239156
97,209393544
98,320417198


In [28]:
ful_result2 = pd.DataFrame()


total_iterations = len(d['TIC'])


for i in tqdm(d['TIC'], total=total_iterations, desc='Processing TICs'):
    query = f"""
    SELECT
  b.main_id,
  b.otype,
  b.otype_txt,
  i.id AS "TIC",
  ot.label AS "Other type"
FROM basic AS b
JOIN ident AS i ON b.oid = i.oidref
JOIN otypedef AS ot ON b.otype = ot.otype
WHERE id = 'TIC{i}'
ORDER BY id;
    """
    result = simbadtap.run_sync(query)
    
    
    ful_result2 = pd.concat([ful_result2, result.to_table().to_pandas()], ignore_index=True)


#ful_result2 = ful_result2.drop(columns=['oidref'])


#ful_result2 = ful_result2[['id'] + [col for col in ful_result2.columns if col != 'id']]
ful_result2

Processing TICs: 100%|████████████████████████| 100/100 [00:06<00:00, 15.15it/s]


,main_id,otype,otype_txt,TIC,Other type
0,EC 12267-1751,HS*,HS*,TIC 5816844,HotSubdwarf
1,PG 1350+372,HS*,HS*,TIC 23746001,HotSubdwarf
2,UCAC4 576-127603,HS*,HS*,TIC 27782233,HotSubdwarf
3,EC 04144-7226,HS*,HS*,TIC 33770579,HotSubdwarf
4,PG 1156-037,HS*,HS*,TIC 35020718,HotSubdwarf
...,...,...,...,...,...
70,PG 1043+760,HS*,HS*,TIC 142491300,HotSubdwarf
71,ATO J331.3394+35.5514,V*,V*,TIC 162239156,Variable*
72,EC 21390-2930,HS*,HS*,TIC 209393544,HotSubdwarf
73,GALEX J075147.0+092526,HS*,HS*,TIC 320417198,HotSubdwarf
